## A notebook to visualize data from the Letterboxed dataset to use in a Movie recomandation model

### Imports

In [3]:
!pip install scikit-surprise

In [4]:


import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import sklearn


from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split




import os

import numpy as np

from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt

import re

### Importing the files

In [6]:
movies = pd.read_csv("C:/Users/ranhe/PycharmProjects/MovieRecommendationML/CSVFiles/movie_data.csv", lineterminator='\n')
ratings = pd.read_csv("C:/Users/ranhe/PycharmProjects/MovieRecommendationML/CSVFiles/ratings_export.csv")

### Heading the data to see how it is setup 

In [8]:
movies.head(5)

,_id,genres,image_url,imdb_id,imdb_link,movie_id,movie_title,original_language,overview,popularity,production_countries,release_date,runtime,spoken_languages,tmdb_id,tmdb_link,vote_average,vote_count,year_released
0,5fc85f606758f69634496fd3,"[""Music"",""Animation""]",film-poster/4/6/4/4/4/0/464440-football-freaks...,NaN,NaN,football-freaks,Football Freaks,en,"Football crazy, football mad. Don’t watch this...",0.600,"[""United Kingdom""]",1971-12-05,0.0,[],535272.0,https://www.themoviedb.org/movie/535272/,0.0,0.0,1971.0
1,5fc85ff26758f696344ace0c,[],film-poster/2/4/5/5/0/0/245500-aftermath-0-230...,tt0586129,http://www.imdb.com/title/tt0586129/maindetails,aftermath-1960,Aftermath,en,Aftermath was the pilot for an unsold TV serie...,0.600,[],1960-04-17,22.0,[],318331.0,https://www.themoviedb.org/movie/318331/,8.0,1.0,1960.0
2,5fc85f606758f69634496fcd,"[""Drama""]",film-poster/9/3/3/1/8/93318-where-chimneys-are...,tt0045731,http://www.imdb.com/title/tt0045731/maindetails,where-chimneys-are-seen,Where Chimneys Are Seen,ja,Gosho’s most celebrated film both in Japan and...,1.568,"[""Japan""]",1953-03-05,108.0,"[""日本語""]",117779.0,https://www.themoviedb.org/movie/117779/,6.6,10.0,1953.0
3,5fc85f606758f69634496fd1,"[""Drama""]",NaN,tt0187327,http://www.imdb.com/title/tt0187327/maindetails,the-musicians-daughter,The Musician's Daughter,en,Carl Wagner's good wife was dying. His heart b...,0.600,"[""United States of America""]",1911-12-12,15.0,[],560377.0,https://www.themoviedb.org/movie/560377/,0.0,0.0,1911.0
4,5fc85f606758f69634496fd4,"[""Documentary""]",film-poster/4/5/4/6/0/3/454603-50-years-of-fab...,tt4769914,http://www.imdb.com/title/tt4769914/maindetails,50-years-of-fabulous,50 Years of Fabulous,en,50 Years of Fabulous recounts the rich history...,0.600,[],2018-05-17,75.0,[],525187.0,https://www.themoviedb.org/movie/525187/,0.0,0.0,2018.0


In [9]:
movies.shape

(285504, 19)

In [10]:
movies.isnull().sum()

_id                         0
genres                  10632
image_url               21097
imdb_id                 41702
imdb_link               41702
movie_id                    2
movie_title              2165
original_language       10632
overview                24256
popularity              10632
production_countries    10632
release_date            14454
runtime                 15449
spoken_languages        10632
tmdb_id                  5587
tmdb_link                5587
vote_average            10632
vote_count              10632
year_released            7572
dtype: int64

In [11]:
movies.describe()

,popularity,runtime,tmdb_id,vote_average,vote_count,year_released
count,274872.000000,270055.000000,279917.000000,274872.000000,274872.000000,277932.000000
mean,2.965787,66.386236,350433.142535,4.121326,61.627558,1992.439438
std,29.765843,142.899320,243316.810867,3.002508,547.102640,29.153126
min,0.000000,0.000000,2.000000,0.000000,0.000000,1874.000000
25%,0.600000,16.000000,134117.000000,0.000000,0.000000,1976.000000
50%,0.871000,80.000000,327033.000000,5.100000,2.000000,2005.000000
75%,2.004000,96.000000,522637.000000,6.400000,8.000000,2015.000000
max,6023.329000,51420.000000,952462.000000,10.000000,30211.000000,2028.000000


In [12]:
ratings.head(5)

,_id,movie_id,rating_val,user_id
0,5fc57c5d6758f6963451a07f,feast-2014,7,deathproof
1,5fc57c5d6758f6963451a063,loving-2016,7,deathproof
2,5fc57c5d6758f6963451a0ef,scripted-content,7,deathproof
3,5fc57c5d6758f6963451a060,the-future,4,deathproof
4,5fc57c5c6758f69634519398,mank,5,deathproof


In [16]:
ratings.shape

(11078167, 4)

### Cleaning data


In [19]:
# Removing duplicate rows
movies.drop_duplicates(inplace=True)
ratings.drop_duplicates(inplace=True)

# Removing missing values
movies.dropna(inplace=True)
ratings.dropna(inplace=True)

In [21]:
ratings.isnull().sum()

_id           0
movie_id      0
rating_val    0
user_id       0
dtype: int64

In [23]:
ratings.columns

Index(['_id', 'movie_id', 'rating_val', 'user_id'], dtype='object')

In [24]:
ratings.shape

(11078161, 4)

## Starting with building the model

In [27]:

# Define a reader with the appropriate scale for your ratings
reader = Reader(rating_scale=(1, 10))  # Adjust based on your rating scale

# Load the data from Pandas
data = Dataset.load_from_df(ratings[['user_id', 'movie_id', 'rating_val']], reader)

# Train-test split
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Use SVD for matrix factorization
algo = SVD()
algo.fit(trainset)

# Test on the testset
predictions = algo.test(testset)


In [28]:
from surprise import accuracy

# Compute and print RMSE
rmse = accuracy.rmse(predictions)

# Compute and print MAE
mae = accuracy.mae(predictions)


RMSE: 1.4058
MAE:  1.0654


In [29]:
for prediction in predictions[:5]:  # Adjust the slice for more samples
    print(prediction)


user: mrmoviesrd item: the-negotiator r_ui = 7.00   est = 7.03   {'was_impossible': False}
user: karmstrongkta item: the-holiday-calendar r_ui = 1.00   est = 5.05   {'was_impossible': False}
user: nickdenbow item: marry-me-2022 r_ui = 5.00   est = 4.50   {'was_impossible': False}
user: moements   item: gifted-2017 r_ui = 7.00   est = 7.49   {'was_impossible': False}
user: bulgar     item: no-good-deed-2014 r_ui = 3.00   est = 4.04   {'was_impossible': False}


In [30]:
from collections import defaultdict

def get_top_n(predictions, n=10):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    
    # Sort predictions for each user and retrieve top N
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    
    return top_n

top_n = get_top_n(predictions, n=10)
for uid, user_ratings in top_n.items():
    print(f"User {uid}: {user_ratings}")


User mrmoviesrd: [('the-shawshank-redemption', 9.484684408192496), ('parasite-2019', 9.398582364870782), ('cowboy-bebop', 9.117537897067912), ('the-lord-of-the-rings-the-return-of-the-king', 8.999128593342213), ('the-shining', 8.991153542515972), ('the-thing', 8.959900992623908), ('there-will-be-blood', 8.896198291926115), ('evil-dead-ii', 8.881568296466048), ('terminator-2-judgment-day', 8.86737489072985), ('i-saw-the-devil', 8.844129885956578)]
User karmstrongkta: [('the-battle-of-algiers', 10), ('kitbull', 9.510644926253786), ('bringing-up-baby', 9.465022834238399), ('f-for-fake', 9.459566113427769), ('oj-made-in-america', 9.204548159854486), ('the-night-of-the-hunter', 9.13416969094298), ('inside-out-2015', 9.133643303866114), ('laura', 9.108002962158043), ('princess-mononoke', 8.854236853174134), ('the-princess-and-the-frog', 8.793628960683733)]
User nickdenbow: [('before-sunset', 9.408503534161394), ('scream', 8.85851616002438), ('clueless', 8.801494570090346), ('in-the-mood-for-